In [2]:
import os
import re
import subprocess
import json

In [3]:
with open("benchmark_list", 'r') as f:
    files = f.read().split()

for file in files:
    # print(file)
    with open(f'./{file}', 'r') as f:
        lines = f.readlines()
    
    params = {}
    kernel_name = ''
    header_lines = ['/* params start */\n', '#define ITERATIONS 10000\n']
    i = 0
    print_start = 0
    param_start = -1
    while i < len(lines):
        if '/* start param define */' in lines[i]:
            lines[i:i] = ['#include <omp.h>\n', '#define ceild(n,d)  ceil(((double)(n))/((double)(d)))\n', '#define floord(n,d) floor(((double)(n))/((double)(d)))\n', '#define max(x,y) ((x) > (y)? (x) : (y))\n', '#define min(x,y) ((x) < (y)? (x) : (y))\n', '\n']
            i += 6
            param_start = i
        elif param_start + 1 and 'end parameters define' in lines[i]:
            k = 0
            for j in range(param_start + 1, i):
                # print(lines[j])
                output = re.search(r'#\s*define\s+([0-9a-zA-Z_]+)\s+(\d+\.\d+|\d+)', lines[j])
                # print(output.group(2))
                params[output.group(1)] = int(output.group(2))
                k += 1
            param_start = -1
        elif f'/* start kernel func */' in lines[i]:
            kernel_name = re.search(r'void\s([a-zA-z_0-9]+)\(', lines[i+1]).group(1)
            # print(kernel_name)
            header_params = re.findall(r'int ([^,\)]+)', lines[i+1])
            # print(header_params)
            i += 1
        elif '#pragma scop' in lines[i]:
            lines.insert(i, '    double time_start = omp_get_wtime();\n')
            i += 1
        elif '#pragma endscop' in lines[i]:
            lines.insert(i + 1, '    printf("%f\\n", time_end - time_start);\n')
            lines.insert(i + 1, '    double time_end = omp_get_wtime();\n')
            i += 2
        elif kernel_name and kernel_name in lines[i]:
            output = re.search(r'\((.*)\)', lines[i])
            kernel_params = output.group(1).split(', ')
            for j in range(len(header_params)):
                header_lines.append(f'#define {header_params[j]} {params[kernel_params[-len(header_params) + j]]}\n')
               
        i += 1
    
    # print(params)
    
    
    header_lines.append('/* params end */\n')
    # print("".join(header_lines))
    with open(f'./{file[:-2]}.check.h', 'w') as f:
        f.writelines(header_lines)
    
    # print("".join(lines))
    with open(f'./{file[:-2]}.check.c', 'w') as f:
        f.writelines(lines)

In [62]:
with open("benchmark_list", 'r') as f:
    files = f.read().split()

for file in files:
    os.remove(f'{file[:-2]}.check.h')

In [52]:
with open("benchmark_list", 'r') as f:
    files = f.read().split()

for file in files:
    # print(file)
    with open(f'./{file}', 'r') as f:
        lines = f.readlines()
    
    add = 0
    i = 0
    while i < len(lines):
        if '#pragma scop' in lines[i] and not lines[i+1] == "for (int iter = 0; iter < ITERATIONS; iter++){\n":
            lines.insert(i+1, 'for (int iter = 0; iter < ITERATIONS; iter++){\n')
            add = 1
            i += 1
        elif '#pragma endscop' in lines[i] and add:
            lines.insert(i, '}\n')
            break

               
        i += 1
    
    
    # print("".join(lines))
    with open(f'{file}', 'w') as f:
        f.writelines(lines)